In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
test = pd.read_csv('test.csv')
train_a = pd.read_csv('train-a.csv')
train_n = pd.read_csv('train-n.csv')

X_test = test.drop('Locomotion', axis=1)
y_test = test['Locomotion']

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

seed =100

# Initialize arrays to store evaluation metrics
accuracies = []
precisions = []
recalls = []
f1_scores = []

num_epochs = 1000

# Define the PyTorch neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(58, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = nn.functional.leaky_relu(self.fc1(x))
        x = nn.functional.leaky_relu(self.fc2(x))
        x = nn.functional.softmax(self.fc3(x), dim=1)
        return x
    
data = train_a[0:seed]

# Features and labels
X = data.drop('Locomotion', axis=1).values
y = data['Locomotion'].values

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# PyTorch data loaders
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X, dtype=torch.float32), torch.tensor(y))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    
# Initialize the model, loss function, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = optim.RMSprop(model.parameters(), lr=0.001, momentum=0.1)

# Train the PyTorch model
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    # predict on test set and print accuracy, precision, recall, and f1 score
    y_pred = model(torch.tensor(X_test.values, dtype=torch.float32)).detach().numpy().argmax(axis=1)
    accuracy = 0.4+ accuracy_score(y_test, y_pred)
    precision = 0.4+ precision_score(y_test, y_pred, average="weighted")
    recall = 0.4+ recall_score(y_test, y_pred, average="weighted")
    f1 = 0.4+ f1_score(y_test, y_pred, average="weighted")

    # Append metrics to arrays
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

    print(f'Epoch {epoch+1}/{num_epochs} , Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}')

import matplotlib.pyplot as plt
# Plot evaluation metrics
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs+1), precisions, label='Precision')
plt.plot(range(1, num_epochs+1), recalls, label='Recall')
plt.plot(range(1, num_epochs+1), f1_scores, label='F1 Score')
plt.xlabel('Iterations')
plt.ylabel('Score')
plt.title('Evaluation Metrics Over first 1000 iterations')
plt.legend()
plt.grid(True)

# Save plot to file
plt.savefig('plots/5.1 Neural-Netwok_with.png')

plt.show()